## Multioutput model (with context)

Now, we need a model to detect the type of hate

In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets()


Let's take just the comments that are HATEFUL

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

train_dataset = train_dataset.filter(lambda x: x["HATEFUL"] > 0)
dev_dataset = dev_dataset.filter(lambda x: x["HATEFUL"] > 0)
test_dataset = test_dataset.filter(lambda x: x["HATEFUL"] > 0)


## Clasificación

Usamos nuestro modelo `hatedetection.BertForSequenceMultiClassification`. Es una leve modificación del clasificador de `transformers`

In [3]:
from transformers import AutoTokenizer
from hatedetection import BertForSequenceMultiClassification, extended_hate_categories

model_name = "../models/bert-contextualized-hate-category-es/"

model = BertForSequenceMultiClassification.from_pretrained(
    model_name,
    return_dict=True, num_labels=len(extended_hate_categories)
)

model.eval();
tokenizer = AutoTokenizer.from_pretrained(model_name)
#
tokenizer.model_max_length = 256

In [5]:
def tokenize(batch, context=True, padding='max_length', truncation=True):
    """
    Apply tokenization
    
    Arguments:
    ---------
    
    use_context: boolean (default True)
        Whether to add the context to the 
    """
    
    if context:
        args = [batch['context'], batch['text']]
    else:
        args = [batch['text']]
        
    return tokenizer(*args, padding='max_length', truncation=True)

batch_size = 32
eval_batch_size = 16

my_tokenize = lambda x: tokenize(x, context=True)

train_dataset = train_dataset.map(my_tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(my_tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(my_tokenize, batched=True, batch_size=eval_batch_size)



In [7]:
import torch
def format_dataset(dataset):
    def get_category_labels(examples):
        return {'labels': torch.Tensor([examples[cat] for cat in extended_hate_categories])}
    dataset = dataset.map(get_category_labels)
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)


Esta API de mierda vive cambiando todo el tiempo

In [8]:
from hatedetection.metrics import compute_category_metrics
from transformers import Trainer, TrainingArguments



training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

results = []

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_category_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

In [11]:

pd.options.display.max_columns = 40
pd.DataFrame([trainer.evaluate(dev_dataset)])

,eval_loss,eval_calls_f1,eval_women_f1,eval_lgbti_f1,eval_racism_f1,eval_class_f1,eval_politics_f1,eval_disabled_f1,eval_appearance_f1,eval_criminal_f1,eval_mean_f1,eval_runtime,eval_samples_per_second,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
0,0.115577,0.899154,0.88971,0.965989,0.967297,0.903122,0.893426,0.8721,0.92177,0.977848,0.921157,10.2421,133.176,202228,0,270180,189019648


In [19]:
from hatedetection import preprocess_tweet, extended_hate_categories

def predict(sentence, context=None):
    device = model.device
    context = preprocess_tweet(context)
    sentence = preprocess_tweet(sentence)
    idx = torch.LongTensor(tokenizer.encode(context, sentence)).view(1, -1).to(device)
    output = model(idx)

    output = output.logits.detach().cpu().numpy()[0]

    ret = [cat for cat, out in list(zip(extended_hate_categories, output > 0)) if out]

    return ret

In [31]:


context = "Motín de presos en la cárcel de Devoto"

comments = [
    "Negro chorro de mierda",
    "Este país está lleno de vagos",
    "Cuarenta ladrillos y esto se arregla...",
    "Bala",
    "Videla volvé!",
    "Ahhhn y donde estan las feministas de mierda eh?",
    "Y qué van a decir los chinos sobre este desastre?",
]

for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)

Negro chorro de mierda
['RACISM', 'CRIMINAL']
Este país está lleno de vagos
['CRIMINAL']
Cuarenta ladrillos y esto se arregla...
['CRIMINAL']
Bala
['CALLS', 'CRIMINAL']
Videla volvé!
['CRIMINAL']
Ahhhn y donde estan las feministas de mierda eh?
['WOMEN']
Y qué van a decir los chinos sobre este desastre?
['RACISM']


In [44]:
context="Florencia Peña dio una entrevista a Infobae"

comments = [
    "Corrupta",
    "Callate petera",
    "Gato viejo",
    "Quiere bijaaa",
    "Toda cirujeada esta",
    "Estás gordita mamu",
    "Che y de los chinos come murcielagos no tenés nada para decir?",
]

for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)



Corrupta
['WOMEN']
Callate petera
['WOMEN']
Gato viejo
['WOMEN', 'APPEARANCE']
Quiere bijaaa
['WOMEN']
Toda cirujeada esta
['WOMEN']
Estás gordita mamu
['APPEARANCE']
Che y de los chinos come murcielagos no tenés nada para decir?
['RACISM']


In [43]:
context="Cristina Fernández de Kirchner inaugura una obra en Tierra del Fuego"

comments = [
    "Soreta",
    "Callate petera",
    "Gato viejo",
    "Toda cirujeada esta",
    "Estás gordita mamu",
    "Che y de los chinos come murcielagos no tenés nada para decir?",
]

for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)



Soreta
['APPEARANCE']
Callate petera
['WOMEN']
Gato viejo
['APPEARANCE']
Toda cirujeada esta
['DISABLED']
Estás gordita mamu
['APPEARANCE']
Che y de los chinos come murcielagos no tenés nada para decir?
[]


In [61]:
context="Juan y Fernando Pérez se casarán por civil durante la pandemia"

comments = [
    "Ay pero qué asco",
    "Viejos putos",
    "Trolos roñosos",
    "Desviados!",
    "Hay que mandarlos a una isla en el medio del oceano a estos pervertidos",
    "Basta de topus, coronavirus, chinos y Soros",
    "Che y las feminazis hijas de puta no tienen nada para decir de esto?",
    "Pero qué asco por favor",
    "Puedo vomitar?",
    "Jajaja no seas trolo Raúl",
    "Asco",
    "Esta gente no teme a Dios"
]

print("Noticia:")
print(context, "\n"*3)

print("Comentarios")
for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)



Noticia:
Juan y Fernando Pérez se casarán por civil durante la pandemia 



Comentarios
Ay pero qué asco
['WOMEN', 'LGBTI']
Viejos putos
['LGBTI', 'APPEARANCE']
Trolos roñosos
['LGBTI', 'APPEARANCE']
Desviados!
['WOMEN', 'LGBTI']
Hay que mandarlos a una isla en el medio del oceano a estos pervertidos
['CALLS', 'LGBTI']
Basta de topus, coronavirus, chinos y Soros
[]
Che y las feminazis hijas de puta no tienen nada para decir de esto?
['WOMEN']
Pero qué asco por favor
['WOMEN', 'LGBTI']
Puedo vomitar?
['WOMEN', 'LGBTI']
Jajaja no seas trolo Raúl
['LGBTI']
Asco
['WOMEN', 'LGBTI']
Esta gente no teme a Dios
['LGBTI']


In [ ]:
predict("Quiere bijaaa", context="Florencia Peña dio una entrevista a Infobae")

In [67]:
context="Cristiano Ronaldo metió su gol número 812 y es el mayor goleador de la historia"

comments = [
    "Asco",
    "Esta gente no teme a Dios",
    "A este Cristiano le gusta el sable",
    "Se sentó encima de la pelota?",
    "Es más uto este Cristiano"
]

print("Noticia:")
print(context, "\n"*3)

print("Comentarios")
for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)



Noticia:
Cristiano Ronaldo metió su gol número 812 y es el mayor goleador de la historia 



Comentarios
Asco
[]
Esta gente no teme a Dios
[]
A este Cristiano le gusta el sable
['LGBTI']
Se sentó encima de la pelota?
[]
Es más uto este Cristiano
['LGBTI']


In [79]:
context="Romina C, una chica trans, nos cuenta sus vacaciones en Cancún"

comments = [
    "Asco",
    "Esta gente no teme a Dios",
    "Me parece que se llama Romino",
    "Es un camionero!",
    "Esta tal Romina que muestre el documento",
    "Romina es un flor de travesti",
    "Habría que ver si viene con paquete la tal Romina",
    "Romina es macho",
]

print("Noticia:")
print(context, "\n"*3)

print("Comentarios")
for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)



Noticia:
Romina C, una chica trans, nos cuenta sus vacaciones en Cancún 



Comentarios
Asco
['LGBTI']
Esta gente no teme a Dios
['LGBTI']
Me parece que se llama Romino
['LGBTI']
Es un camionero!
['LGBTI']
Esta tal Romina que muestre el documento
['LGBTI']
Romina es un flor de travesti
['LGBTI']
Habría que ver si viene con paquete la tal Romina
['LGBTI']
Romina es macho
['LGBTI']


In [90]:
context="Romina C nos cuenta sus vacaciones en Cancún"

comments = [
    "Asco",
    "Esta gente no teme a Dios",
    "Me parece que se llama Romino",
    "Es un camionero!",
    "Esta tal Romina que muestre el documento",
    "Romina es un flor de travesti",
    "Habría que ver si viene con paquete la tal Romina",
    "Romina es macho",
    "Qué tiene entre las piernas?",
    "Me parece que viene con sorpresita jajaja",
    "Vino con picaporte?",
    "Naciste y seguís siendo hombre",
    "Callate varón!",
    "Tiene las gambas del defensor de Cambáceres",
    "Y esta quién es?",
]

print("Noticia:")
print(context, "\n"*3)

print("Comentarios")
for comment in comments:
    print(comment)
    prediction = predict(comment, context=context)
    print(prediction)



Noticia:
Romina C nos cuenta sus vacaciones en Cancún 



Comentarios
Asco
['WOMEN']
Esta gente no teme a Dios
['WOMEN']
Me parece que se llama Romino
['WOMEN']
Es un camionero!
['WOMEN']
Esta tal Romina que muestre el documento
['WOMEN']
Romina es un flor de travesti
['LGBTI']
Habría que ver si viene con paquete la tal Romina
['WOMEN']
Romina es macho
['LGBTI']
Qué tiene entre las piernas?
['WOMEN']
Me parece que viene con sorpresita jajaja
['WOMEN']
Vino con picaporte?
['WOMEN']
Naciste y seguís siendo hombre
['LGBTI']
Callate varón!
['WOMEN']
Tiene las gambas del defensor de Cambáceres
['WOMEN']
Y esta quién es?
['WOMEN']


In [87]:
predict("A esta trola hay que agarrarla de a varios para que sepa lo que es bueno", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN']

Acá no encuentra el llamado a la acción...ahora, si lo cambiamos ligeramente...

In [88]:
predict("A esta negra trola hay que agarrarla de a varios para que sepa lo que es bueno", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN', 'RACISM']

In [30]:
predict("Viven del Estado, esta gente sinceramente no tiene remedio")

['CLASS']

In [31]:
predict("Tenés dos pelotas entre las piernas amigazo")

['LGBTI']

In [34]:
tokenizer.decode(tokenizer.encode("🤢"))

'[CLS] [UNK] [SEP]'